In [ ]:
# Get current position of the robot to plot
def odom_callback(msg):
    
    x_odom.append(msg.pose.pose.position.x)
    y_odom.append(msg.pose.pose.position.y)

In [ ]:
# Get minimum distance from nearest obstacle in front of the robot to plot
def scan_callback(msg):
    
    global min_l, min_c, min_r
    
    min_l = np.min(msg.ranges[0:240])
    min_c = np.min(msg.ranges[240:480])
    min_r = np.min(msg.ranges[480:720])
    
    y_scan.append(min_c)

In [ ]:
# Avoid obstacle while going towards the goal
def auto_control():
    
    min_th = 0.5 # threshold distance from obstacle
    if (min_c<min_th): # obstacle in front
        print("Obstacle! I need to turn")
        vel.linear.x = 0
        vel.angular.z = -1
    else:
        if (min_r<min_th): # obstacle on the right
            print("Obstacle! I need to turn")
            vel.linear.x = 0
            vel.angular.z = 1
        elif (min_l<min_th): # obstacle on the left
            print("Obstacle! I need to turn")
            vel.linear.x = 0
            vel.angular.z = -1
    pub_vel.publish(vel)

In [ ]:
# Directly control the robot via user inputs
def user_control():
    
	b1 = widgets.Button(description='turn left')
	b2 = widgets.Button(description='go forward')
	b3 = widgets.Button(description='turn right')
	b4 = widgets.Button(description='stop here')
	grid = widgets.GridspecLayout(2,3)
	grid[0,1] = b2
	grid[1,0] = b1
	grid[1,1] = b4
	grid[1,2] = b3
	display(grid, output)
        
	def b1_clicked(b):
		vel.linear.x = 0
		vel.angular.z = 1
		pub_vel.publish(vel)
	def b2_clicked(b):
		vel.linear.x = 1
		vel.angular.z = 0
		pub_vel.publish(vel)
	def b3_clicked(b):
		vel.linear.x = 0
		vel.angular.z = -1
		pub_vel.publish(vel)
	def b4_clicked(b):
		vel.linear.x = 0
		vel.angular.z = 0
		pub_vel.publish(vel)
        
	b1.on_click(b1_clicked)
	b2.on_click(b2_clicked)
	b3.on_click(b3_clicked)
	b4.on_click(b4_clicked)

In [ ]:
# Let the robot move autonomously in the environment
def auto_move():
	canc = GoalID()
	
	print("You chose modality 1! \n")
	
	# Ask the user a new target position
	print("Set x and y of new target position: ")
	x_pos = widgets.FloatText(
		value = None,
		description = 'x: ',
		disabled = False
	)
	y_pos = widgets.FloatText(
		value = None,
		description = 'y: ',
		disabled = False
	)
	send_goal = widgets.Button(
		description = 'Send goal',
		disabled = False
	)
	box = widgets.VBox(children=[x_pos, y_pos, send_goal])  
	     
	def publish_target(change):
		target.goal.target_pose.pose.position.x = x_pos.value   
		target.goal.target_pose.pose.position.y = y_pos.value
		pub_goal.publish(target)
	
	display(box, output)
	send_goal.on_click(publish_target)
    
	# Drive the robot towards target
	auto_control()
	
	# Asking the user if target has to be cancelled
	choose_to_canc = widgets.RadioButtons(
		options = ['Yes', 'No'],
		layout = {'max_width': 'max-content'},
		description = 'Do you want to cancel this goal?',
		disabled = False,
		value = None
	)
        
	def cancelling(change):
		if (choose_to_canc.value=='Yes'): # the user wants to cancel it
			pub_canc.publish(canc)
			print("Goal cancelled")
		else:
			print("Ok, going towards the goal")
                
	display(choose_to_canc, output)
	choose_to_canc.observe(cancelling, names=['value'])

In [ ]:
# Drive the robot with keyboard inputs
def drive_alone():
    
	print("You chose modality 2! \n")
	
	print("Use buttons to move the robot:")
	user_control()

In [ ]:
# Drive the robot in the environment while being assisted for obstacle avoidance
def drive_assistance():
    
	print("You chose modality 3! \n")
	
	print("Use buttons to move the robot with assistance:")
	user_control()
	assistance = widgets.Checkbox(
		value = False,
		description = 'Assistance',
		disabled = False
	)
	def enable_assistance(change):
		if(assistance.value==True):
			auto_control()
			print("Assistance enabled")
		else:
			print("Assistance disabled")
        
	display(assistance, output)
	assistance.observe(enable_assistance, names='value')

In [ ]:
# Decide control modality
def ui_decide():
	
	choose_mod = widgets.RadioButtons(
		options = ['robot moves autonomously to goal', 'drive the robot to goal', 'drive the robot with assistance to goal'],
		layout = {'width': 'max-content'},
		description = 'Choose:',
		disabled = False,
		value = None
	)
    
	def choosing(change):
		if (choose_mod.value=='robot moves autonomously to goal'):
			auto_move()
		elif (choose_mod.value=='drive the robot to goal'):
 			drive_alone()
		elif (choose_mod.value=='drive the robot with assistance to goal'):
			drive_assistance()
		else:
			print("This modality doesn't exist.")
        
	display(choose_mod, output)
	choose_mod.observe(choosing, names=['value'])

In [ ]:
# Plot current position of the robot, laser scanner data and targets position
def plot_data():

    # Robot position
    ax_odom.plot(x_odom, y_odom)
    ax_odom.set_title("Robot position")
    ax_odom.set_xlabel("x [m]")
    ax_odom.set_ylabel("y [m]")
    
    # Laser scanner data
    ax_scan.plot(y_scan)
    ax_scan.set_title("Minimum distance from obstacles")
    ax_scan.set_ylabel("distance [m]")
    
    # Target position
    x_target.append(target.goal.target_pose.pose.position.x)
    y_target.append(target.goal.target_pose.pose.position.y)
    ax_target.plot(x_target,y_target,'*')
    ax_target.set_title("Target position")
    ax_target.set_xlabel("x [m]")
    ax_target.set_ylabel("y [m]")
    
    fig.tight_layout()

In [ ]:
# Start the interface node to control the robot
def interface():
    rospy.init_node("user_interface")
    
    global pub_goal
    global pub_canc
    global pub_vel
    
    pub_goal = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size=10)
    pub_canc = rospy.Publisher("/move_base/cancel", GoalID, queue_size=10)
    pub_vel = rospy.Publisher("/cmd_vel", Twist, queue_size=10)
    sub_odom = rospy.Subscriber("/odom", Odometry, odom_callback)
    sub_scan = rospy.Subscriber("/scan", LaserScan, scan_callback)
    
    ui_decide()
    plot_data()

In [ ]:
import sys
sys.path.append('/root/ros_ws/src/RT1-III_assignment')
import final_assignment
import rospy
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist
from move_base_msgs.msg import MoveBaseActionGoal
from actionlib_msgs.msg import GoalID

# Define needed variables 
target = MoveBaseActionGoal()
vel = Twist()

output = widgets.Output()

# Initialize matplotlib widget to plot data
%matplotlib widget
global x_odom, y_odom, y_scan, x_target, y_target
x_odom = []
y_odom = []
y_scan = []
x_target = []
y_target = []
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(9,5))
ax_odom, ax_scan, ax_target = axes.flatten()
plt.show()

# Start the interface
interface()